In [ ]:
# ====== 설정 ======
from pathlib import Path
import numpy as np
import open3d as o3d 

# 경로
BASE_DIR = Path(r"..\assets\ply\shaft")
INPUT_DIR     = BASE_DIR / "ply"          # PLY 읽기 폴더
OUTPUT_DIR    = BASE_DIR / "xyz"          # 변환된 XYZ 저장 폴더
MERGE_IN_DIR  = OUTPUT_DIR                # 병합 입력(.xyz) 폴더
MERGE_OUT_DIR = BASE_DIR / "xyz_merged"   # 병합 결과 저장 폴더

# 파라미7S
N_PER_PLY     = 10_000    # 각 PLY → XYZ 변환 시 정확히 뽑을 포인트 수
SUFFIX        = "_point"  # 변환 파일명 접미사

np.random.seed(42)

# ====== 유틸 ======
def next_serial_filename(out_dir: Path, prefix="xyz_data_", ext=".xyz", width=3) -> Path:
    out_dir.mkdir(parents=True, exist_ok=True)
    i = 1
    while True:
        p = out_dir / f"{prefix}{i:0{width}d}{ext}"
        if not p.exists():
            return p
        i += 1

def ensure_exact_n(points: np.ndarray, n: int, seed: int = 42) -> np.ndarray:
    """points에서 정확히 n개 반환 (부족하면 복원샘플링으로 채움, 많으면 비복원 다운샘플)."""
    rng = np.random.default_rng(seed)
    pts = np.asarray(points, dtype=float)
    if pts.ndim == 1:
        if pts.size % 3 != 0:
            raise ValueError("포인트 형식 오류 (길이 % 3 != 0)")
        pts = pts.reshape(-1, 3)
    if pts.shape[1] > 3:
        pts = pts[:, :3]

    m = len(pts)
    if m == 0:
        raise ValueError("입력 포인트가 0개입니다.")

    if m == n:
        return pts
    if m > n:
        idx = rng.choice(m, n, replace=False)
        return pts[idx]
    # m < n → 복원샘플링으로 채움
    add_idx = rng.choice(m, n - m, replace=True)
    return np.vstack([pts, pts[add_idx]])

# ====== 변환 함수 ======
def convert_ply_to_xyz(ply_path: Path, out_dir: Path, n_points: int = 10_000, suffix: str = "_point"):
    """
    단일 .ply → 점군(.xyz) 저장
    - 삼각형 메쉬면 Poisson 디스크 샘플링
    - 포인트클라우드면 랜덤 샘플링
    - 결과는 항상 정확히 n_points개로 맞춤
    """
    # 1) 메쉬로 시도
    pcd_pts = None
    mesh = o3d.io.read_triangle_mesh(str(ply_path))
    if mesh is not None and mesh.has_vertices():
        try:
            if mesh.has_triangles():
                pcd = mesh.sample_points_poisson_disk(number_of_points=int(n_points))
                pcd_pts = np.asarray(pcd.points)
        except Exception as e:
            print(f"[WARN] 메쉬 샘플링 실패 → 포인트클라우드로 재시도: {ply_path.name} ({e})")

    # 2) 포인트클라우드로 시도
    if pcd_pts is None:
        pc = o3d.io.read_point_cloud(str(ply_path))
        pts = np.asarray(pc.points)
        if pts.size == 0:
            print(f"[ERR] 포인트가 없습니다: {ply_path}")
            return None
        pcd_pts = pts

    # 항상 정확히 n_points로 보정
    pts_n = ensure_exact_n(pcd_pts, n_points, seed=42)

    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f"{ply_path.stem}{suffix}.xyz"
    np.savetxt(out_path, pts_n, fmt="%.6f")
    print(f"[OK] {ply_path.name} → {out_path} (N={len(pts_n)})")
    return out_path

# ====== 일괄 변환 ======
def batch_convert(input_dir: Path, output_dir: Path, n_points: int = 10_000, suffix: str = "_point"):
    if not input_dir.is_dir():
        print(f"[ERR] 입력 폴더가 존재하지 않습니다: {input_dir}")
        return
    ply_files = sorted(list(input_dir.glob("*.ply")))
    if not ply_files:
        print(f"[INFO] PLY 파일이 없습니다: {input_dir}")
        return

    cnt = 0
    for ply in ply_files:
        if convert_ply_to_xyz(ply, output_dir, n_points, suffix) is not None:
            cnt += 1
    print(f"[DONE] 총 {cnt}/{len(ply_files)} 개 파일 변환 완료.  (in: {input_dir} → out: {output_dir})")


# ====== xyz 병합: 모든 포인트 '전부' 합침 (다운샘플 없음) ======
def merge_xyz_and_save(input_dir: Path,
                       output_dir: Path,
                       n_per_file: int,
                       prefix: str = "xyz_data_",
                       ext: str = ".xyz",
                       width: int = 3):
    """
    input_dir 안의 *.xyz를 모두 읽어 그대로 병합한 뒤,
    output_dir에 시리얼 파일명(예: xyz_data_001.xyz)으로 저장.
    - 각 입력 .xyz는 (N,3) 형식이어야 하며, N == n_per_file 아니면 경고만 출력.
    - 반환: 저장 경로(Path) 또는 None (입력 없음)
    """
    xyz_files = sorted(input_dir.glob("*.xyz"))
    if not xyz_files:
        print(f"[INFO] {input_dir} 안에 .xyz 파일이 없습니다.")
        return None

    merged_list = []
    total_expected = len(xyz_files) * n_per_file
    for f in xyz_files:
        data = np.loadtxt(f)
        if data.ndim == 1:
            if data.size % 3 != 0:
                print(f"[WARN] {f.name} 포인트 형식 오류 → 스킵")
                continue
            data = data.reshape(-1, 3)
        if data.shape[1] > 3:
            data = data[:, :3]
        n = len(data)
        if n != n_per_file:
            print(f"[WARN] {f.name}: N={n} (기대 {n_per_file})")
        merged_list.append(data)
        print(f"[READ] {f.name} (N={n})")

    if not merged_list:
        print("[INFO] 병합할 데이터가 없습니다.")
        return None

    merged_xyz = np.vstack(merged_list)
    out_path = next_serial_filename(output_dir, prefix=prefix, ext=ext, width=width)
    np.savetxt(out_path, merged_xyz, fmt="%.6f")
    print(f"[DONE] {len(xyz_files)}개 파일 병합 → {out_path} "
          f"(총 포인트: {len(merged_xyz):,d}, 기대치≈{total_expected:,d})")
    return out_path



In [ ]:
# 120회 반복: 매번 PLY→XYZ 변환 후 병합 저장
for _ in range(120):
    batch_convert(INPUT_DIR, OUTPUT_DIR, N_PER_PLY, SUFFIX)
    merge_xyz_and_save(MERGE_IN_DIR, MERGE_OUT_DIR, N_PER_PLY,
                       prefix="xyz_data_", ext=".xyz", width=3)